In [2]:
    
#!/usr/bin/env python

import nltk, os, subprocess, code, glob, re, traceback, sys, inspect
from time import clock, sleep
from pprint import pprint
import json
import zipfile
# import ner

#from convertRtfToText import convertRtfToText




In [65]:

def preprocess(document):
    
    try:
        
        lines = [el.strip() for el in document.split("\n") if len(el) > 0]  # Splitting on the basis of newlines 
        lines = [nltk.word_tokenize(el) for el in lines]    # Tokenize the individual lines
        lines = [nltk.pos_tag(el) for el in lines]  # Tag them
     
        sentences = nltk.sent_tokenize(document)    # Split/Tokenize into sentences (List of strings)
        sentences = [nltk.word_tokenize(sent) for sent in sentences]    # Split/Tokenize sentences into words (List of lists of strings)
        tokens = sentences
        sentences = [nltk.pos_tag(sent) for sent in sentences]    # Tag the tokens - list of lists of tuples - each tuple is (<word>, <tag>)
       
        dummy = []
        for el in tokens:
            dummy += el
        tokens = dummy
        
        return tokens, lines, sentences
    except Exception as e:
        print(e) 

def tokenize(inputString):
    try:
        tokens, lines, sentences = preprocess(inputString)
        return tokens, lines, sentences
    except Exception as e:
        print(e)

def getEmail(inputString, infoDict, debug=False): 
   
    email = None
    try:
        pattern = re.compile(r'\S*@\S*')
        matches = pattern.findall(inputString) # Gets all email addresses as a list
        email = matches
    except Exception as e:
        print(e)

    infoDict['email'] = email

    if debug:
        print("\n", pprint(infoDict), "\n")
        code.interact(local=locals())
    return email

def getPhone(inputString, infoDict, debug=False):
  

    number = None
    try:
        pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
            
        match = pattern.findall(inputString)
        
        match = [re.sub(r'[,.]', '', el) for el in match if len(re.sub(r'[()\-.,\s+]', '', el))>6]
            
        match = [re.sub(r'\D$', '', el).strip() for el in match]
            
        match = [el for el in match if len(re.sub(r'\D','',el)) <= 15]
            
        try:
            for el in list(match):
                
                if len(el.split('-')) > 3: continue # Year format YYYY-MM-DD
                for x in el.split("-"):
                    try:
                  
                        if x.strip()[-4:].isdigit():
                            if int(x.strip()[-4:]) in range(1900, 2100):
                            
                                match.remove(el)
                    except:
                        pass
        except:
            pass
        number = match
    except:
        pass

    infoDict['phone'] = number

    if debug:
        print("\n", pprint(infoDict), "\n")
        code.interact(local=locals())
    return number

def getName(inputString, infoDict, debug=False):
   

    
    indianNames = open("all_Names.txt", "r").read().lower()
 
    indianNames = set(indianNames.split())


    otherNameHits = []
    nameHits = []
    name = None

    try:
        tokens, lines, sentences = tokenize(inputString)
       
        grammar = r'NAME: {<NN.*><NN.*><NN.*>*}'
  
        chunkParser = nltk.RegexpParser(grammar)
        all_chunked_tokens = []
        for tagged_tokens in lines:
           
            if len(tagged_tokens) == 0: continue # Prevent it from printing warnings
            chunked_tokens = chunkParser.parse(tagged_tokens)
            all_chunked_tokens.append(chunked_tokens)
            for subtree in chunked_tokens.subtrees():
           
                if subtree.label() == 'NAME':
                    for ind, leaf in enumerate(subtree.leaves()):
                        if leaf[0].lower() in indianNames and 'NN' in leaf[1]:
                          
                            hit = " ".join([el[0] for el in subtree.leaves()[ind:ind+3]])
                          
                            if re.compile(r'[\d,:]').search(hit): continue
                            nameHits.append(hit)
        if len(nameHits) > 0:
            nameHits = [re.sub(r'[^a-zA-Z \-]', '', el).strip() for el in nameHits] 
            name = " ".join([el[0].upper()+el[1:].lower() for el in nameHits[0].split() if len(el)>0])
            otherNameHits = nameHits[1:]

    except Exception as e:
        print(traceback.format_exc())
        print(e)         

    infoDict['name'] = name
    #infoDict['otherNameHits'] = otherNameHits

    if debug:
        print("\n", pprint(infoDict), "\n")
        code.interact(local=locals())
    return name, otherNameHits  


In [75]:
nlist = []
plist = []
elist = []
names=""
others=""
number=""
email=""
import os
for file in os.listdir("C:\\Users\\Abhinav\\Desktop\\ml\\resumes"):
    infodict={}
    if file.endswith(".txt"):
        files = open(file,"r")
        content = ""
        for i in files.readlines():
            content+=i
        names,others = getName(content,infodict)
        email = getEmail(content,infodict)
        number = getPhone(content,infodict)
        nlist.append(infodict["name"])
        plist.append(infodict["phone"])
        elist.append(infodict["email"])
        

In [84]:
nplist = []
for i in range(len(plist)):
    if(len(plist[i])>0):
        nplist.append(plist[i][0])
    else:
        nplist.append("111-111-1111")

In [86]:
eplist = []
for i in range(len(elist)):
    if(len(elist[i])>0):
        eplist.append(elist[i][0])
    else:
        eplist.append("abc@xyz.com")

In [87]:
eplist

['Abbaskhurram00001@gmail.com',
 'abc@xyz.com',
 'amarnath000001@gmail.com',
 'arupkumar00001@gmail.com',
 '|tv00001@gmail.com',
 'abc@xyz.com']

In [88]:
ninfodict = {}
ninfodict["name"] = nlist
ninfodict["phone"] = nplist
ninfodict["email"] = eplist

In [89]:
ninfodict


{'email': ['Abbaskhurram00001@gmail.com',
  'abc@xyz.com',
  'amarnath000001@gmail.com',
  'arupkumar00001@gmail.com',
  '|tv00001@gmail.com',
  'abc@xyz.com'],
 'name': ['Abbas Khurram',
  'Sridharamurty Aabha Aabharana',
  'Dash Board',
  'Arup Kumar',
  'Thomas',
  'Gangadhar Vasan'],
 'phone': ['929-111-1111',
  '111-111-1111',
  '999-123-1234',
  '888-111-1234',
  '1-222-111-1111',
  '111-111-1111']}

In [90]:
import json

def writeToJSONFile(path, fileName, data):
    filePathNameWExt = './' + path + '/' + fileName + '.json'
    with open(filePathNameWExt, 'w') as fp:
        json.dump(data, fp)
writeToJSONFile('./','submissions',ninfodict)